In [19]:
# pip install --quiet openai

In [20]:
# pip install --quiet  PyPDF2

In [21]:
# pip install --upgrade --quiet  langchain

In [22]:
# pip install --quiet langchain-openai faiss-cpu tiktoken

In [23]:
# pip install --quiet unstructured

In [24]:
# pip install --quiet  pdf2image

In [25]:
# pip install --quiet  pdfminer

In [69]:
import os
from openai import OpenAI

In [27]:
# import os
# from dotenv import load_dotenv
# load_dotenv()
# api_key= os.getenv("OPENAI_API_KEY")

In [70]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [71]:
import PyPDF2

In [30]:
pip install pypdf

Note: you may need to restart the kernel to use updated packages.


In [73]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from operator import itemgetter
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings

from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS


In [74]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [75]:
import os
import re
from PyPDF2 import PdfReader
from langchain.docstore.document import Document

def load_pdf(file_path, start_page=10):
    with open(file_path, 'rb') as file:
        pdf_reader = PdfReader(file)
        text = ''
        for page_number, page in enumerate(pdf_reader.pages):
            if page_number + 1 >= start_page:  # Start reading from the 10th page
                text += page.extract_text()
        # Remove emails from the text
        text_without_emails = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '', text)
        # Ensure text is Unicode supported
        unicode_text = text_without_emails.encode('utf-8', 'ignore').decode('utf-8')
        return [Document(page_content=unicode_text, metadata={'file_name': os.path.basename(file_path)})]

pdf_directory = "Must-Read-Books-for-Financial-Professionals"  # Replace this with the path to your directory containing PDF files
pdf_files = [os.path.join(pdf_directory, file) for file in os.listdir(pdf_directory) if file.endswith('.pdf')]

docs = []
for pdf_file in pdf_files:
    docs.extend(load_pdf(pdf_file))



In [76]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 2000, chunk_overlap=0)

In [77]:
text = " "
for i in docs:
    text = text + "\n" + i.page_content

In [78]:

chunks_splitted = text_splitter.create_documents([text])

In [79]:
len(chunks_splitted)

6537

In [67]:
import torch
from transformers import AutoModel, AutoTokenizer
from scipy.spatial.distance import cosine

# Get our models - The package will take care of downloading the models automatically
# For best performance: Muennighoff/SGPT-5.8B-weightedmean-nli-bitfit
tokenizer = AutoTokenizer.from_pretrained("Muennighoff/SGPT-125M-weightedmean-nli-bitfit")
model = AutoModel.from_pretrained("Muennighoff/SGPT-125M-weightedmean-nli-bitfit")





In [87]:
def embeddings(text,model= model , tokenizer= tokenizer):
    batch_tokens = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        last_hidden_state = model(**batch_tokens, output_hidden_states=True, return_dict=True).last_hidden_state
    weights = (
    torch.arange(start=1, end=last_hidden_state.shape[1] + 1)
    .unsqueeze(0)
    .unsqueeze(-1)
    .expand(last_hidden_state.size())
    .float().to(last_hidden_state.device))
    input_mask_expanded = (
    batch_tokens["attention_mask"]
    .unsqueeze(-1)
    .expand(last_hidden_state.size())
    .float())
    sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded * weights, dim=1)
    sum_mask = torch.sum(input_mask_expanded * weights, dim=1)
    
    embeddings = sum_embeddings / sum_mask
    return embeddings.detach().cpu().numpy()[0].tolist()
    

In [88]:
trial = chunks_splitted[0:10]

In [99]:
p=[]
c=1
for i in chunks_splitted:
    x={"id": str(c), "values": embeddings(i.page_content), "metadata":{"text": i.page_content}}
    p.append(x)
    c=c+1 

In [6]:
# import dotenv
# from dotenv import load_dotenv
# load_dotenv()
# from pinecone import Pinecone
# pinecone_api_key = os.getenv("pinecone_api_key")
# pc = Pinecone(api_key = pinecone_api_key)
# index = pc.Index("finance")

In [ ]:
from load_pinecone_index import index 
def upsert_data_in_pinecone(dict, index):
    index.upsert(vectors=[dict], namespace="Fianl_set1")

for i in range(len(p)):
    upsert_data_in_pinecone(p[i], index= index)

In [1]:
import cohere
from dotenv import load_dotenv
import os
load_dotenv()
cohere_api_key= os.getenv("cohere_api_key")
co = cohere.Client(cohere_api_key)

In [10]:
from embedding import embeddings
from load_pinecone_index import index
def retriever_from_pinecone_then_reranking(query, index = index, embeddings= embeddings):
    embed_query = embeddings(query)
    pc = index.query(vector = embed_query,top_k=5, include_values = False, include_metadata = True)
    id_from_docs =[]
    text_retrieved =[]
    for i in pc["matches"]:
        id_from_docs.append(i["id"])
    for i in pc["matches"]:
        text_retrieved.append(i["metadata"]["text"])
    rerank_docs = co.rerank(query=query, documents=text_retrieved, top_n=2, model="rerank-english-v2.0")
    
    id_lists={}
    id_lists["Context1"] = id_from_docs[rerank_docs[0].index]
    id_lists["Context2"] = id_from_docs[rerank_docs[1].index]
    contexts = {}
    contexts["Context1"] = rerank_docs[0].document["text"]
    contexts["Context2"] = rerank_docs[1].document["text"]
    contexts_score={}
    contexts_score["Context1"] = rerank_docs[0].relevance_score
    contexts_score["Context2"] = rerank_docs[1].relevance_score
    return contexts, contexts_score, id_lists 
    
    

In [8]:
import time
def retrieve_from_pinecone_then_summarizer(query):
    embed_query = embeddings(query)
    pc = index.query(vector = embed_query,top_k=5, include_values = False, include_metadata = True)
    id_from_docs =[]
    text_retrieved =[]
    for i in pc["matches"]:
        id_from_docs.append(i["id"])
    for i in pc["matches"]:
        text_retrieved.append(i["metadata"]["text"])
    text = ""
    for i in text_retrieved:
        text= text+ "\n" + i
    response = co.summarize(
    text=text,
    model='command',
    length='medium',
    extractiveness='medium')
    summary = response.summary
    return summary 

In [35]:
px = embeddings("What is random walk and how is it useful in finance")

In [32]:
x=list(px)

In [37]:
# retriever_from_pinecone_then_reranking("What is random walk and how is it useful in finance")
pc = index.query(vector = [px],top_k=5, include_values = False, include_metadata = True)

AttributeError: 'numpy.ndarray' object has no attribute '_composed_schemas'